# Try HUAP in 5 Minutes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mircus/HUAP/blob/main/notebooks/Try_HUAP.ipynb)

<p align="center">
  <img src="https://raw.githubusercontent.com/Mircus/HUAP/main/HUAP-logo.png" alt="HUAP Logo" width="400"/>
</p>

**HUman Agentic Platform** — trace-first primitives for deterministic, testable agents.

This notebook walks you through HUAP’s core features using **stub mode** — no API keys needed, fully deterministic.

| What you’ll see | How |
|---|---|
| Hello World pipeline | `huap demo` |
| 5-node flagship demo | `huap flagship` (research → analyze → gate → synthesize → memorize) |
| Agent CI drift detection | `huap flagship --drift` + `huap ci run` |

In [ ]:
# Install HUAP from PyPI
!pip install -q huap-core

# Stub mode: deterministic responses, no API keys
import os
os.environ["HUAP_LLM_MODE"] = "stub"

---
## Step 1: Hello World

The simplest HUAP pipeline: a 2-node graph that echoes a message.

`huap demo` runs the graph, records a JSONL trace, and generates an HTML report.

In [ ]:
!huap demo --no-open

# --- Show trace events ---
import json

print("\n" + "=" * 60)
print("Trace events:")
print("=" * 60)
with open("huap_demo/hello.jsonl") as f:
    for line in f:
        event = json.loads(line)
        kind = event.get("event", event.get("type", "?"))
        node = event.get("node", "")
        print(f"  {kind:20s} {node}")

# --- Display HTML report inline ---
from IPython.display import HTML, display

print("\n" + "=" * 60)
print("HTML Report (inline):")
print("=" * 60)
display(HTML(open("huap_demo/hello.html").read()))

---
## Step 2: The Flagship Demo

A 5-node pipeline showcasing the full HUAP stack:

1. **Research** — gather information on a topic
2. **Analyze** — extract key findings
3. **Human Gate** — checkpoint for human review
4. **Synthesize** — produce a polished memo
5. **Memorize** — store findings for future recall

All in stub mode — no API keys, fully deterministic.

In [ ]:
!huap flagship --no-open

# --- Display the generated memo ---
from pathlib import Path
from IPython.display import Markdown, HTML, display

memo = Path("huap_flagship_demo/memo.md")
if memo.exists():
    print("\n" + "=" * 60)
    print("Generated Memo:")
    print("=" * 60)
    display(Markdown(memo.read_text()))

# --- Display HTML trace report inline ---
report = Path("huap_flagship_demo/trace.html")
if report.exists():
    print("\n" + "=" * 60)
    print("Trace Report (inline):")
    print("=" * 60)
    display(HTML(report.read_text()))

---
## Step 3: Agent CI — Catch Drift

HUAP’s killer feature: **deterministic CI for agents**.

1. We run the flagship demo with `--drift` to inject a controlled change
2. Then we run `huap ci run` against a golden baseline
3. The CI runner detects the behavioral drift and reports it

This is how you gate agent regressions in a real CI pipeline.

In [ ]:
# Run with drift injected
!huap flagship --drift --no-open

# Run CI suite against golden baseline
!huap ci run suites/smoke/suite.yaml --html reports/ci_report.html 2>&1 || true

# --- Show results ---
from pathlib import Path
from IPython.display import HTML, display

report = Path("reports/ci_report.html")
if report.exists():
    print("=" * 60)
    print("CI Report (inline):")
    print("=" * 60)
    display(HTML(report.read_text()))
else:
    print("\nCI report not generated (expected if suites/smoke not bundled with pip install).")
    print("In a real repo, the CI runner diffs traces against golden baselines")
    print("and produces a PASS/FAIL report with detailed event-level diffs.")

# Show the diff report if it was generated
diff_report = Path("huap_flagship_demo/diff.html")
if diff_report.exists():
    print("\n" + "=" * 60)
    print("Drift Diff Report:")
    print("=" * 60)
    display(HTML(diff_report.read_text()))

---
## What’s Next?

You just ran HUAP’s core workflow: **trace → replay → diff → CI gate**.

### Learn more
- [GitHub](https://github.com/Mircus/HUAP) — source code, examples, contributing guide
- [PyPI](https://pypi.org/project/huap-core/) — `pip install huap-core`
- [Getting Started](https://github.com/Mircus/HUAP/blob/main/GETTING_STARTED.md) — full tutorial

### Try with your own agents
```bash
# Wrap any script as a traced agent
huap trace wrap --out traces/my_agent.jsonl -- python my_agent.py

# Or use the LangChain adapter
from hu_core.adapters.langchain import HuapCallbackHandler
handler = HuapCallbackHandler(out="traces/langchain.jsonl")
chain.invoke({"input": "hello"}, config={"callbacks": [handler]})
```

### Key CLI commands
```bash
huap init myproject          # Scaffold a new project
huap trace run pod graph.yaml # Run and record
huap trace diff a.jsonl b.jsonl # Compare two runs
huap ci run suite.yaml       # CI gate with baselines
huap trace report trace.jsonl # Shareable HTML report
```